# Previously On AdaBoost
Which the train the weak learner on mistake made by the previous weak learner.This is done by giving the weight on each observation ,more weight for the misclassification.So that next weak learner will focus on the misclassification observation.


# Key Ideas
1. Base idea to train the weak learner .**Learning from the mistake** this key idea on the Sequential Ensemble.
2. How do you focus of observation that is misclassified? **by giving the weight to each observation proportional to the misclassification**



# Generalizing the Ideas
1. **Loss Function:** Measure how well the model is fitting each training example in the dataset.
2. **Gradient of Loss Function:** For the single observation this called the *residual*.
3. **Residual:** Measure the amount of misclassification. Residual here and the *weights* in the Adaboost having the similar measurement.

This give birth to Idea we can use different loss and its gradient loss as way to tell upcoming weak learner **which observation you need to focus.**

This idea we are calling as the **Gradient Boosting.**

# Gradient Boosting

## Gradient Descent for minimization

In [29]:
import matplotlib
import numpy as np
import matplotlib.pyplot as plt

matplotlib.rcParams["backend"] = "nbAgg"
matplotlib.rcParams["figure.figsize"] = (4,3)
matplotlib.get_backend()

'nbAgg'

### Visual Branin

In [30]:
def branin(a, b, c, r, s, t, w1, w2):
    return a * (w2 - (b * np.square(w1)) + (c * w1) - r) ** 2 + s * (1 - t) * np.cos(w1) + s


vec_branin = np.vectorize(branin)

In [31]:
a, b, c, r, s, t = 1, 5.1 / (4 * np.pi ** 2), 5 / np.pi, 6, 10, 1 / (8 * np.pi)

w1_min, w1_max, w2_min, w2_max = -5, 18, -10, 18
w1_mesh, w2_mesh = np.meshgrid(np.linspace(w1_min, w1_max, 400),
                               np.linspace(w2_min, w2_max, 400))
z_mesh = vec_branin(a, b, c, r, s, t, w1_mesh, w2_mesh)

plt.figure(figsize=(8, 4))
ax = plt.subplot(121, projection="3d")
plt.setp(ax, xlabel="$w_1$", ylabel="$w_2$", zlabel="$f(w_1,w_2)$", xlim=(w1_min, w1_max), ylim=(w2_min, w2_max),
         zlim=(-50, 400), position=[0.025, 0.15, 0.5, 0.9])
ax.plot_surface(w1_mesh, w2_mesh, z_mesh, alpha=0.6, lw=0.25, edgecolors="k", rstride=20, cstride=20,
                cmap="viridis")
ax.view_init(elev=25.0, azim=-100.0)
plt.contour(w1_mesh, w2_mesh, z_mesh, zdir="z", levels=np.array([1, 2, 4, 7, 12, 20, 50, 75, 125, 175]),
            offset=-50, alpha=0.5, cmap="viridis")

ax = plt.subplot(122)
ctr = ax.contour(w1_mesh, w2_mesh, z_mesh, levels=np.array([1, 2, 4, 7, 12, 20, 50, 75, 125, 175]),
                 alpha=0.5, cmap="viridis")
ax.clabel(ctr, inline=1, fontsize=6)
ax.set_xlabel("$w_1$")
ax.set_ylabel("$w_2$")
plt.tight_layout()
plt.show()

<IPython.core.display.Javascript object>

### Find the local minimum point

> **Brute Force Way:** we can find over grid (w1 and w2) values and apply the function on those we can find the minimum point.
> If the grid is Coarse then we may mix the optimal point
> if the grid is too fine we have to search large space.

In [32]:
w1_min, w1_max, w2_min, w2_max = -5, 18, -10, 18
w1_mesh, w2_mesh = np.meshgrid(np.linspace(w1_min, w1_max, 400),
                               np.linspace(w2_min, w2_max, 400))
z_mesh = vec_branin(a, b, c, r, s, t, w1_mesh, w2_mesh)
plt.figure(figsize=(8, 4))
ax = plt.subplot(121)
plt.setp(ax, title="Large/Coarser Grid ")
plt.contour(w1_mesh, w2_mesh, z_mesh, levels=np.array([1, 2, 4, 7, 12, 20, 50, 75, 125, 175]),
            alpha=0.5, cmap="viridis")
w1_mesh, w2_mesh = np.meshgrid(np.linspace(w1_min, w1_max, 10),
                               np.linspace(w2_min, w2_max, 10))
plt.scatter(w1_mesh, w2_mesh)

w1_mesh, w2_mesh = np.meshgrid(np.linspace(w1_min, w1_max, 400),
                               np.linspace(w2_min, w2_max, 400))
ax = plt.subplot(122)
plt.setp(ax, title="Fine Grid ")
plt.contour(w1_mesh, w2_mesh, z_mesh, levels=np.array([1, 2, 4, 7, 12, 20, 50, 75, 125, 175]),
            alpha=0.5, cmap="viridis")
w1_mesh, w2_mesh = np.meshgrid(np.linspace(w1_min, w1_max, 20),
                               np.linspace(w2_min, w2_max, 20))
plt.scatter(w1_mesh, w2_mesh)
plt.tight_layout()
plt.show()

<IPython.core.display.Javascript object>

Second approach :  We walk along the slope of the function (gradient)
1. Gradient point int he direction of the steepest ascent.
2. So we have walk positive direction, so negate the gradient to walk opposite direction.

In [33]:
from scipy.optimize import line_search


def gradient_descent(func, grad, start, max_iter=100, args=()):
    n_iter = 0
    x_cur, x_next = np.array(start), None
    descent_path = np.full(shape=(max_iter + 1, 2), fill_value=np.nan)
    descent_path[n_iter] = x_cur

    while True:
        n_iter += 1
        # walk opposite direction
        neg_gradient = -grad(x_cur, *args)
        direction = neg_gradient / np.linalg.norm(neg_gradient)

        # find the step
        step = line_search(func, grad, x_cur, direction, args=args)

        if step[0] is None:
            distance = 1.0
        else:
            distance = step[0]

        # update the position
        x_next = x_cur + distance * direction
        descent_path[n_iter] = x_next

        err = np.linalg.norm(x_next - x_cur)
        if err <= 1e-3 or n_iter >= max_iter:
            break

        x_cur = x_next

    return x_next, descent_path

In [34]:
def branin(w, a, b, c, r, s, t):
    return a * (w[1] - b * w[0] ** 2 + c * w[0] - r) ** 2 + s * (1 - t) * np.cos(w[0]) + s


def branin_gradient(w, a, b, c, r, s, t):
    return np.array([2 * a * (w[1] - b * w[0] ** 2 + c * w[0] - r) * (-2 * b * w[0] + c) - s * (1 - t) * np.sin(w[0]),
                     2 * a * (w[1] - b * w[0] ** 2 + c * w[0] - r)])

In [35]:
w_init = np.array([-4, -5])
a, b, c, r, s, t = 1, 5.1 / (4 * np.pi ** 2), 5 / np.pi, 6, 10, 1 / (8 * np.pi)
w_optimal, w_path = gradient_descent(branin, branin_gradient, w_init, args=(a, b, c, r, s, t))

In [36]:
w1_min, w1_max, w2_min, w2_max = -5, 18, -10, 18
w1_mesh, w2_mesh = np.meshgrid(np.linspace(w1_min, w1_max, 400),
                               np.linspace(w2_min, w2_max, 400))
z_mesh = vec_branin(a, b, c, r, s, t, w1_mesh, w2_mesh)

# visual the gradient path
plt.figure(figsize=(8, 4))
ax = plt.subplot(121)
plt.setp(ax, xlim=(-5, 18), ylim=(-10, 18), title="Gradient descent")
ax.contour(w1_mesh, w2_mesh, z_mesh, levels=np.array([1, 2, 4, 7, 12, 20, 50, 75, 125, 175]),
           alpha=0.5, cmap="viridis")
ax.plot(w_path[:, 0], w_path[:, 1])
ax.scatter(w_init[0], w_init[1])
ax.scatter(w_optimal[0], w_optimal[1], zorder=2)

ax = plt.subplot(122)
plt.setp(ax, xlim=(1.5, 5.5), ylim=(-1, 3), title="Gradient descent (Zoomed in)")
ax.contour(w1_mesh, w2_mesh, z_mesh, levels=np.array([1, 2, 4, 7, 12, 20, 50, 75, 125, 175]),
           alpha=0.5, cmap="viridis")
ax.plot(w_path[:, 0], w_path[:, 1])
ax.scatter(w_init[0], w_init[1])
ax.scatter(w_optimal[0], w_optimal[1], zorder=2)
plt.tight_layout()
plt.show()

<IPython.core.display.Javascript object>

3. Gradient descent typically demonstrates zig-zagging behavior, especially in narrow valleys. This is a consequence of rapidly changing gradient direction. The magnitude of the gradient also becomes smaller as we approach the minima.

4. Of the four minima, which one will gradient descent converge to? That depends on the initial guess. As we see below, different initializations will cause gradient descent to reach different minim

In [37]:
w1_min, w1_max, w2_min, w2_max = -5, 18, -10, 18
w1_mesh, w2_mesh = np.meshgrid(np.linspace(w1_min, w1_max, 400),
                               np.linspace(w2_min, w2_max, 400))
z_mesh = vec_branin(a, b, c, r, s, t, w1_mesh, w2_mesh)

n = 20
inits = np.r_[np.c_[np.random.uniform(0, 15, n), np.random.uniform(10, 16, n)],
              np.c_[np.random.uniform(-5, 0, n), np.random.uniform(-10, 0, n)],
              np.c_[np.random.uniform(12, 17, n), np.random.uniform(-10, 0, n)]]
ax = plt.subplot(111)
plt.setp(ax, xlim=(-5, 18), ylim=(-10, 18), title="Gradient descent with different Initialization")
ax.contour(w1_mesh, w2_mesh, z_mesh, levels=np.array([1, 2, 4, 7, 12, 20, 50, 75, 125, 175]),
           alpha=0.5, cmap="viridis")

for w_init in inits:
    w, path = gradient_descent(branin, branin_gradient, w_init, args=(a, b, c, r, s, t))
    plt.plot(path[:, 0], path[:, 1], c="r", lw=1)
ax.scatter(inits[:, 0], inits[:, 1])
plt.show()

C:\Users\Lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\optimize\_linesearch.py:305: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)


<IPython.core.display.Javascript object>

### Classification problem

In [38]:
from sklearn.datasets import make_blobs

X, y = make_blobs(n_samples=200, n_features=2, centers=[[-1.5, -1.5], [1.5, 1.5]]
                  , random_state=42)
plt.scatter(X[:, 0], X[:, 1], c=y)
plt.show()

<IPython.core.display.Javascript object>

In [39]:
def square_loss(w, X, y):
    return 0.5 * np.sum(np.square(y - np.dot(X, w)))


def square_loss_grad(w, X, y):
    return -np.dot(X.T, (y - np.dot(X, w)))

In [40]:
plt.figure(figsize=(9, 4))

# visual the surface and the contours

# calculating the gird and its value.
w1_min, w1_max, w2_min, w2_max = -1, 1, -1, 1
w1_mesh, w2_mesh = np.meshgrid(np.linspace(w1_min, w1_max, 100),
                               np.linspace(w2_min, w2_max, 100))
z_mesh = np.apply_along_axis(square_loss,
                             1,
                             np.c_[w1_mesh.ravel(), w2_mesh.ravel()],
                             X, y)
z_mesh = np.reshape(z_mesh, newshape=w1_mesh.shape)

# plotting the surface
ax = plt.subplot(121, projection="3d")
plt.setp(ax, title="Loss func surface", xlim=(w1_min, w2_max), ylim=(w2_min, w2_max))
ax.plot_surface(w1_mesh, w2_mesh, z_mesh, alpha=0.5, lw=0.25, edgecolors="k", cmap="viridis", rstride=5, cstride=5)
ax.view_init(elev=34, azim=-40.0)
ax.contour(w1_mesh, w2_mesh, z_mesh, levels=np.array([50, 100, 150, 200, 300, 400, 600, 800, 1000]),
           cmap="viridis", alpha=0.5, zdir="z", offset=-50)

#  plotting the contour
ax = plt.subplot(122)
plt.setp(ax, title="Loss function contour")
ctr = ax.contour(w1_mesh, w2_mesh, z_mesh, levels=np.array([50, 100, 150, 200, 300, 400, 600, 800, 1000]),
                 cmap="viridis", alpha=0.5)
ax.clabel(ctr, inline=1, fontsize=6)

plt.tight_layout()
plt.show()

<IPython.core.display.Javascript object>

In [41]:
plt.figure(figsize=(9, 4))
# visual the gradient path
w_init = np.array([-0.0, -0.85])
w_optimum, w_path = gradient_descent(square_loss, square_loss_grad, w_init, args=(X, y))

# build the mesh
w1_min, w1_max, w2_min, w2_max = -1, 1, -1, 1
w1_mesh, w2_mesh = np.meshgrid(np.linspace(w1_min, w1_max, 100),
                               np.linspace(w2_min, w2_max, 100))
z_mesh = np.apply_along_axis(square_loss,
                             1,
                             np.c_[w1_mesh.ravel(), w2_mesh.ravel()],
                             X, y)
z_mesh = np.reshape(z_mesh, newshape=w1_mesh.shape)

# view the contour plot
ax = plt.subplot(121)
plt.setp(ax, title="Gradient Path", xlabel="$w_1$", ylabel="$w_2$")
ax.contour(w1_mesh, w2_mesh, z_mesh, levels=np.linspace(50, 1000, 15), cmap="viridis")
ax.plot(w_path[:, 0], w_path[:, 1], c="magenta")
ax.scatter(w_init[0], w_init[1], c="magenta")
ax.scatter(w_optimum[0], w_optimum[1], c="r")

# view the decision boundary
esp = 0.25
ax = plt.subplot(122)
plt.setp(ax, title='Decision Boundary', xlabel="$x_1$", ylabel="$x_2$")
x1_min, x1_max, x2_min, x2_max = X[:, 0].min() - esp, X[:, 0].max() + esp, X[:, 1].min() - esp, X[:, 1].max() + esp
x1_mesh, x2_mesh = np.meshgrid(np.linspace(x1_min, x1_max, 100),
                               np.linspace(x2_min, x2_max, 100))
z_mesh = np.dot(np.c_[x1_mesh.ravel(), x2_mesh.ravel()], w_optimum)
z_mesh = np.reshape(z_mesh, newshape=x1_mesh.shape)
ax.contourf(x1_mesh, x2_mesh, z_mesh, cmap="RdBu")
ax.contour(x1_mesh, x2_mesh, z_mesh, levels=[0], colors="lime")
ax.scatter(X[:, 0], X[:, 1], c=y, edgecolors="k")

plt.tight_layout()
plt.show()

<IPython.core.display.Javascript object>

In [42]:
print(w_optimum)

[0.13541487 0.1393232 ]


## Gradient Descent + Boosting

1. Adaboost and Gradient Descent is much similar, but they are not same
2. Key difference between them , how they focus the misclassified point in the next iteration by the upcoming weak learner.
3. AdaBoost use the *Weighing the observation proportional to accuracy*
4. Gradient Boosting use the **residual**.

### Intuition

In [43]:
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
import numpy as np

X, y = make_moons(n_samples=200, noise=0.15, random_state=13)
y = 2 * y - 1

In [44]:
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots
from sklearn.tree import DecisionTreeRegressor
from scipy.optimize import minimize_scalar

pio.templates.default = "plotly_white"

# building the grid
esp = 0.25
x_min, x_max = X[:, 0].min() - esp, X[:, 0].max() + esp
y_min, y_max = X[:, 1].min() - esp, X[:, 1].max() + esp
x_mesh, y_mesh = np.meshgrid(np.linspace(x_min, x_max, 100),
                             np.linspace(y_min, y_max, 100))
grid = np.c_[x_mesh.ravel(), y_mesh.ravel()]

# Initialise
n_samples = X.shape[0]
n_estimators = 10
F = np.full(shape=(n_samples,), fill_value=0.0)
ensemble = []

for t in range(n_estimators):
    # calculate the negative residual
    residuals = y - F

    # use the stumps as the weak leaner
    stumps = DecisionTreeRegressor(max_depth=1)
    # train the weak leaner with residual
    stumps.fit(X, residuals)

    # compute the step that produce the best improvement in th loss
    h_reg = stumps.predict(X)
    loss = lambda a: np.linalg.norm(y - (F + a * h_reg)) ** 2
    step = minimize_scalar(loss, method="golden")
    a = step.x

    # update the model
    F += a * h_reg
    ensemble.append((a, stumps))

    # visualize the plot using matplotlib

    if t in [0, 1, 2, 9]:
        plt.figure(figsize=(6, 8))
        plt_idx = [0, 1, 2, 9].index(t)
        ax = plt.subplot(221, projection="3d")
        plt.setp(ax, title=f"Decision Surface Weak Learner")
        z_mesh_wl = stumps.predict(grid)
        ax.plot_surface(x_mesh, y_mesh, z_mesh_wl.reshape(x_mesh.shape), cmap="RdBu", alpha=0.5)
        ax.scatter(X[:, 0], X[:, 1], residuals, c=y, cmap="RdBu", alpha=0.5, edgecolors="k")

        ax = plt.subplot(222)
        plt.setp(ax, title=f'Decision Boundary Weak Learner')
        z_mesh_bound_wl = 2 * (z_mesh_wl >= 0).astype(int) - 1
        ax.contourf(x_mesh, y_mesh, z_mesh_bound_wl.reshape(x_mesh.shape), cmap="RdBu", alpha=0.5)
        ax.scatter(X[:, 0], X[:, 1], c=y, cmap="RdBu", alpha=0.5, edgecolors="k")

        ax = plt.subplot(223, projection="3d")
        plt.setp(ax, title=f'Decision Surface Ensemble')
        z_mesh_ensemble = np.zeros((grid.shape[0],))
        for a, h in ensemble:
            z_mesh_ensemble += a * h.predict(grid)
        ax.plot_surface(x_mesh, y_mesh, z_mesh_ensemble.reshape(x_mesh.shape), cmap="RdBu", alpha=0.5, )
        ax.scatter(X[:, 0], X[:, 1], residuals, c=y, cmap="RdBu", alpha=0.5, edgecolors="k")

        ax = plt.subplot(224)
        plt.setp(ax, title=f'Decision Boundary Ensemble')
        z_mesh_bound_ensemble = 2 * (z_mesh_ensemble >= 0).astype(int) - 1
        ax.contourf(x_mesh, y_mesh, z_mesh_bound_ensemble.reshape(x_mesh.shape), cmap="RdBu", alpha=0.5, )
        ax.scatter(X[:, 0], X[:, 1], c=y, cmap="RdBu", alpha=0.5, edgecolors="k")
        plt.suptitle(f"Iteration {plt_idx}")
        plt.tight_layout()
        plt.show()

    # #visualise the plots using the plotly making slow
    # if t in [0, 1, 9]:
    #     fig = make_subplots(rows=1, cols=2,
    #                         specs=[[{"type": "scene"}, {"type": "scene"}]],
    #                         subplot_titles=("Weak Learner",
    #                                         "Ensemble"),
    #                         row_heights=[600],
    #                         column_widths=[600, 600],
    #                         horizontal_spacing=0.1,
    #                         vertical_spacing=0.1)
    #     # scatter
    #     scatter_marker = go.scatter3d.Marker(colorscale="RdBu", showscale=False, size=5, opacity=0.5,
    #                                          color=y)
    #     scatter_3d_ = go.Scatter3d(x=X[:, 0], y=X[:, 1], z=residuals, mode="markers",
    #                                marker=scatter_marker)
    #     # Surface of weak leaner
    #     grid = np.c_[x_mesh.ravel(), y_mesh.ravel()]
    #     z_mesh_wl = stumps.predict(grid)
    #     d_surface_wl_ = go.Surface(x=x_mesh, y=y_mesh, z=z_mesh_wl.reshape(x_mesh.shape),
    #                                colorscale="RdBu", showscale=False, opacity=0.5)
    #
    #     # Surface of Ensemble
    #     z_mesh_ensemble = np.zeros((grid.shape[0],))
    #     for a, h in ensemble:
    #         z_mesh_ensemble += a * h.predict(grid)
    #
    #     d_surface_ensemble = go.Surface(x=x_mesh, y=y_mesh, z=z_mesh_ensemble.reshape(x_mesh.shape),
    #                                     colorscale="RdBu", showscale=False, opacity=0.5)
    #
    #     # Boundary of weak learner
    #     z_mesh_bound_wl = 2 * (z_mesh_wl >= 0).astype(int) - 1
    #     d_bound_wl = go.Contour(z=z_mesh_bound_wl.reshape(x_mesh.shape),
    #                             colorscale="RdBu", showscale=False, opacity=0.5)
    #
    #     # boundary for ensemble
    #     z_mesh_bound_ensemble = 2 * (z_mesh_ensemble >= 0).astype(int) - 1
    #     d_bound_ensemble = go.Contour(z=z_mesh_bound_ensemble.reshape(x_mesh.shape),
    #                                   colorscale="RdBu", showscale=False, opacity=0.5)
    #
    #     fig.add_traces([d_surface_wl_,
    #                     d_surface_ensemble],
    #                    rows=[1, 1], cols=[1, 2])
    #     fig.update_layout(height=300, width=600, title_text=f"Iteration {t}",
    #                       margin=dict(l=0,r=0,b=0,t=0))
    #     fig.show()
plt.show()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Implementing the Gradient Boosting

In [45]:
from sklearn.tree import DecisionTreeRegressor
from scipy.optimize import minimize_scalar


def fit_gradient_boosting(X, y, n_estimators=10):
    # initialise
    n_samples, n_features = X.shape
    estimators = []
    F = np.full((n_samples,), 0.0)

    for t in range(n_estimators):
        # calculate the residual
        residual = y - F
        # create stumps as the weak learner
        stumps = DecisionTreeRegressor(max_depth=1)
        # train the weak learner with X and residual
        stumps.fit(X, residual)
        # Find the loss
        h_reg = stumps.predict(X)
        # calculate the step
        loss = lambda a: np.linalg.norm(y - (F + a * h_reg)) ** 2
        step = minimize_scalar(loss, method="golden")
        a = step.x

        # update the model
        F += a * h_reg

        estimators.append((a, stumps))

    return estimators

In [46]:
def predict_gradient_boosting(X, estimators):
    pred = np.zeros(X.shape[0])

    for a, h in estimators:
        pred += a * h.predict(X)

    return np.sign(pred)

In [47]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state=11)
estimators = fit_gradient_boosting(X_train, y_train)
y_pred = predict_gradient_boosting(X_test, estimators)


In [48]:
from sklearn.metrics import accuracy_score

test_error = 1 - accuracy_score(y_test, y_pred)
test_error

0.06000000000000005

### Gradient Boosting in sklearn

**Two Key Difference Between the Adaboost and Gradient Boosting.**
1. base learner : Adaboost we can use `any base estimator`,but the gradient boosting we have choose the regression as estimator, most preferred is **DecisionTreeRegression**
2. Loss Function : Adaboost only use the **exponential loss** but in the gradient boosting we cna use any loss function.


In [49]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

X, y = load_breast_cancer(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=124)

ensemble = GradientBoostingClassifier(max_depth=1, n_estimators=20, learning_rate=0.75, random_state=42)
ensemble.fit(X_train, y_train)

GradientBoostingClassifier(learning_rate=0.75, max_depth=1, n_estimators=20,
                           random_state=42)

In [50]:
from sklearn.metrics import accuracy_score

y_pred = ensemble.predict(X_test)
test_error = 1 - accuracy_score(y_pred, y_test)
test_error

0.04195804195804198

A key limitation of GradientBoostingClassifier, however, is its speed; while effective, it does tend to be rather slow on large data sets.

The efficiency bottleneck, as it turns out, is in tree learning. Recall that gradient boosting has to learn a regression tree at each iteration as a base estimator. For large data sets, the number of  splits a tree-learner has to consider becomes prohibitively large.

This has led to the emergence of histogram-based gradient boosting, which aims to speed up base estimator tree learning, allowing gradient boosting to scale up to large data sets

# Histogram-based Gradient Boosting

n_observation = 10**6
n_features = 100

In Decision tree we sort the feature values ,and we will enumerate all possible splits and then we will the best split while have low loss.In this we have to search the large space (10**6  * 100 = 10 **8)

Alternative way we will now construct the bin for each variable.Suppose say 100 bin , now search spce is reduced to (100 * 100 = 10 ** 4)

There is trade off between speed vs accuracy, we gain speed over the accuracy.

However in big data , there will be large amount of redundancy information, so binning the values wont affect much.

In [51]:
from scipy.special import expit

# generate the 1d data as the feature value
n_samples = 50
X = np.random.uniform(low=-6, high=6, size=(n_samples,))

# we sort the data inorder to find the best split
X = np.sort(X)
# target will as sigmoid
y = expit(X)
plt.scatter(X, y)
plt.title("1d data")
plt.show()

<IPython.core.display.Javascript object>

In [52]:
plt.figure(figsize=(4, 3))
best = [np.inf, np.inf]
for i in range(n_samples - 1):
    split = (X[i] + X[i + 1]) / 2
    loss = np.mean((y[X <= split] - np.mean(y[X <= split])) ** 2) + np.mean((y[X > split] - np.mean(y[X > split])) ** 2)
    if loss <= best[1]:
        best = [split, loss]
    plt.vlines(x=split, ymin=0, ymax=1, lw=1, colors="r")
plt.vlines(x=best[0], ymin=0, ymax=1, lw=3, colors="k", linestyles="--")
plt.scatter(X, y)
plt.title(f"Standard : {n_samples - 1} splits to consider")
plt.show()

<IPython.core.display.Javascript object>

In [53]:
def get_colors(color_map,n_colors):
    cmap_ = matplotlib.cm.get_cmap(color_map)
    colors_rgb = cmap_(np.linspace(0,1,n_colors))
    colors_hex = [matplotlib.colors.rgb2hex(c_) for c_ in colors_rgb]
    return colors_hex


In [54]:
plt.figure(figsize=(4, 3))
best = [-np.inf, np.inf]
n_bins = 5
cm = get_colors("rainbow",n_bins+1)
splits =np.interp(np.linspace(0,n_samples,n_bins+1),np.arange(n_samples),X)
for i,split in enumerate(splits):
    plt.vlines(split,ymin=0,ymax=1,colors="r",lw=1)
    plt.axvspan(splits[i-1],splits[i],facecolor=cm[i],alpha=0.5)
    loss = np.mean((y[X <= split] - np.mean(y[X <= split])) ** 2) + np.mean((y[X > split] - np.mean(y[X > split])) ** 2)
    if loss <= best[1]:
        best = [split, loss]
plt.vlines(x=best[0], ymin=0, ymax=1, lw=3, colors="k", linestyles="--")
plt.scatter(X, y)
plt.title(f"Histogram : {n_bins} splits to consider")
plt.show()

C:\Users\Lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


<IPython.core.display.Javascript object>

In [55]:
X, y = load_breast_cancer(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=124)



In [56]:
from sklearn.ensemble import HistGradientBoostingClassifier
ensemble = HistGradientBoostingClassifier(max_depth=2,
                                          max_iter=20,
                                          learning_rate=0.75)
ensemble.fit(X_train,y_train)

y_pred = ensemble.predict(X_test)
test_error = 1 - accuracy_score(y_test,y_pred)
test_error

0.027972027972028024